In [1]:
import pandas as pd
import pickle

# Creating a DataFrame
path = 'dataset/loan_dataset.csv'
df_original = pd.read_csv(path)
df_original.head()

df_original = pd.DataFrame(df_original)

# Filtering the specified columns
filtered_df = df_original[[
    'Employment_Status', 'Annual_Income', 'Credit_Score',  'Outstanding_Debt', 'Loan_Amount_Requested', 
    'Loan_Type','Loan_Purpose', 'Loan_Approval_Status'
]]

filtered_df

,Employment_Status,Annual_Income,Credit_Score,Outstanding_Debt,Loan_Amount_Requested,Loan_Type,Loan_Purpose,Loan_Approval_Status
0,Employed,139901,743,19822,24535,Secured,Home,1
1,Employed,21162,468,18321,8288,Unsecured,Home,0
2,Self-Employed,27815,389,13851,10308,Secured,Vehicle,0
3,Self-Employed,137853,778,28608,33937,Secured,Personal,1
4,Employed,81753,752,8780,23360,Unsecured,Home,1
...,...,...,...,...,...,...,...,...
51995,Self-Employed,56085,381,11909,15723,Secured,Home,1
51996,Self-Employed,79062,726,24696,21209,Unsecured,Vehicle,1
51997,Unemployed,57471,396,12367,10540,Unsecured,Education,0
51998,Employed,49495,362,13240,16765,Secured,Vehicle,0


In [ ]:
# Filter on home loan applications
filtered_df_home = filtered_df[filtered_df['Loan_Purpose'] == 'Home']
df = filtered_df_home.drop(columns=['Loan_Purpose'])    # drop column, as it no longer is necessary
df

,Employment_Status,Annual_Income,Credit_Score,Outstanding_Debt,Loan_Amount_Requested,Loan_Type,Loan_Approval_Status
0,Employed,139901,743,19822,24535,Secured,1
1,Employed,21162,468,18321,8288,Unsecured,0
4,Employed,81753,752,8780,23360,Unsecured,1
8,Self-Employed,121626,803,13940,20239,Secured,0
10,Self-Employed,54670,772,22104,20000,Unsecured,1
...,...,...,...,...,...,...,...
51986,Self-Employed,105630,734,14841,26616,Unsecured,1
51989,Unemployed,118085,714,9803,30183,Secured,0
51990,Self-Employed,134581,756,15172,24717,Unsecured,1
51992,Unemployed,34019,572,19031,8383,Unsecured,0


In [ ]:
nan_count = df.isna().sum()
print("NaN Count:\n", nan_count)

# save under datasets
with open('dataset/filtered_df.pkl', 'wb') as f:
    pickle.dump(df, f)

NaN Count:
 Employment_Status        0
Annual_Income            0
Credit_Score             0
Outstanding_Debt         0
Loan_Amount_Requested    0
Loan_Type                0
Loan_Approval_Status     0
dtype: int64
